# **Procesamiento de Datos a Gran Escala**

## Análisis de fútbol con PySpark

*Autores: Pablo López Perez y Daniel Beteta Francisco*

# Objetivo

Hemos elegido el dataset "results.csv" extraído de la plataforma Kaggle, que en resumidas cuentas nos da información básica sobre el rendimiento de los equipos de la Premier League desde la temporada 2006/2007 a la 2017/2018.

De tal forma, el objetivo de este ejercicio opcional es extraer información sobre dichos equipos jugando como locales.

Así, se van a hacer principalmente 3 transformaciones:
  1.  **Recuento total de puntos a lo largo de las distintas temporadas**: va a ser posible gracias al método map para seleccionar las columnas deseadas, la variable global "DICT_RESULTS_POINTS" para transformar los datos de los resultados de discretos a continuos, y el método reduceByKey para sumar los puntos de cada partido de cada equipo que jugaba como local.
  2. **Recuento de temporadas en primera y porcentaje de victorias, empates y derrotas**: va a ser posible gracias al uso del método groupByKey y a la función "count_hda" pasada como parámetro a map para contabilizar las nombradas métricas.
  3. **Porcentaje de goles a favor vs en contra jugando como local**: va a ser posible gracias al método map para transformar de string a float los datos de los goles a favor y en contra, para luego con el método reduceByKey agregar dicha información, y por último, sacar la proporción mediante map.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!pip install -q pyspark

     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 45.0 MB/s 


In [ ]:
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

from google.colab import drive
from tabulate import tabulate
from pyspark.sql import SparkSession

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
APP_NAME = "PDGE-tutorialSpark1"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
sc = spark.sparkContext

In [ ]:
drive.mount('/content/gdrive', force_remount=True)
path_to_dataset = "/content/gdrive/My Drive/PGDE/results.csv"
rdd_results_csv = sc.textFile(path_to_dataset)

Mounted at /content/gdrive


In [ ]:
rdd_results_list = rdd_results_csv.map(lambda x: x.split(",")).filter(lambda x: x[0] != "home_team")
rdd_results_list.cache()
print(rdd_results_list.take(10))

[['Sheffield United', 'Liverpool', '1.0', '1.0', 'D', '2006-2007'], ['Arsenal', 'Aston Villa', '1.0', '1.0', 'D', '2006-2007'], ['Everton', 'Watford', '2.0', '1.0', 'H', '2006-2007'], ['Newcastle United', 'Wigan Athletic', '2.0', '1.0', 'H', '2006-2007'], ['Portsmouth', 'Blackburn Rovers', '3.0', '0.0', 'H', '2006-2007'], ['Reading', 'Middlesbrough', '3.0', '2.0', 'H', '2006-2007'], ['West Ham United', 'Charlton Athletic', '3.0', '1.0', 'H', '2006-2007'], ['Bolton Wanderers', 'Tottenham Hotspur', '2.0', '0.0', 'H', '2006-2007'], ['Manchester United', 'Fulham', '5.0', '1.0', 'H', '2006-2007'], ['Chelsea', 'Manchester City', '3.0', '0.0', 'H', '2006-2007']]


In [ ]:
DICT_RESULTS_POINTS = {
    "H": 3,
    "D": 1,
    "A": 0,
}
  
def count_hda(x):
  half_season = 38/2
  res_dict = {
      3: 0,
      1: 0,
      0: 0,
  }
  for result in list(x[1]):
    if result in res_dict:
      res_dict[result] += 1

  h, d, a = res_dict.values()
  amount_of_matches = h+d+a

  return (x[0], [amount_of_matches/half_season,
                 round(h/amount_of_matches, 2)*100, 
                 round(d/amount_of_matches, 2)*100, 
                 round(a/amount_of_matches, 2)*100])

rdd_results_by_local_team = rdd_results_list.map(lambda x: (x[0], DICT_RESULTS_POINTS[x[4]]))
rdd_results_points_by_local_team = rdd_results_by_local_team.reduceByKey(lambda x, y: x+y)
rdd_results_amount_hda = rdd_results_by_local_team.groupByKey().map(count_hda)

In [ ]:
rdd_results_by_local_team = rdd_results_list.map(lambda x: (x[0], [float(x[2]), float(x[3])]))
rdd_results_goals = rdd_results_by_local_team.reduceByKey(lambda x, y: [x[0] + y[0], x[1] + y[1]]).map(lambda x: (x[0], [round(x[1][0]/x[1][1], 2)]))

print(rdd_results_goals.take(10))

[('Newcastle United', [1.09]), ('Reading', [0.91]), ('Bolton Wanderers', [1.0]), ('Manchester United', [3.09]), ('Chelsea', [2.64]), ('Aston Villa', [1.0]), ('Manchester City', [2.61]), ('Fulham', [1.08]), ('Middlesbrough', [1.02]), ('Wigan Athletic', [0.77])]


In [ ]:
amount_of_diferent_teams = rdd_results_list.map(lambda x: x[0]).distinct().count()
rdd_results_output = rdd_results_points_by_local_team.join(rdd_results_amount_hda) \
                                                     .join(rdd_results_goals) \
                                                     .map(lambda x: [x[0], x[1][0][0], x[1][0][1][0], x[1][0][1][1], x[1][0][1][2], x[1][0][1][3], x[1][1][0]])

rdd_results_output_ordered = rdd_results_output.takeOrdered(amount_of_diferent_teams, lambda x: -x[1])                                                            
print(rdd_results_output_ordered)

[['Manchester United', 544, 12.0, 75.0, 14.000000000000002, 11.0, 3.09], ['Chelsea', 511, 12.0, 67.0, 23.0, 10.0, 2.64], ['Arsenal', 506, 12.0, 67.0, 22.0, 11.0, 2.59], ['Manchester City', 505, 12.0, 68.0, 16.0, 15.0, 2.61], ['Liverpool', 473, 12.0, 60.0, 28.999999999999996, 12.0, 2.56], ['Tottenham Hotspur', 464, 12.0, 61.0, 22.0, 18.0, 1.93], ['Everton', 422, 12.0, 53.0, 26.0, 21.0, 1.66], ['West Ham United', 303, 11.0, 40.0, 24.0, 35.0, 1.02], ['Stoke City', 297, 10.0, 43.0, 28.000000000000004, 28.999999999999996, 1.16], ['Newcastle United', 280, 10.0, 40.0, 27.0, 33.0, 1.09], ['Aston Villa', 247, 10.0, 33.0, 32.0, 35.0, 1.0], ['Sunderland', 238, 10.0, 32.0, 28.999999999999996, 39.0, 0.94], ['Fulham', 228, 8.0, 42.0, 24.0, 34.0, 1.08], ['West Bromwich Albion', 222, 9.0, 35.0, 26.0, 39.0, 0.94], ['Swansea City', 190, 7.0, 38.0, 28.000000000000004, 34.0, 1.05], ['Southampton', 171, 6.0, 40.0, 28.999999999999996, 31.0, 1.33], ['Blackburn Rovers', 169, 6.0, 40.0, 27.0, 32.0, 1.16], ['Wi

In [ ]:
headers = ["Team", "Total amount of points", 
           "Number of seasons in the Premier League",
           "% of victories as local",
           "% of draws as local",
           "% of losses as local",
           "% of goals scored vs conceded as local"]

print(tabulate(rdd_results_output_ordered, headers=headers, showindex=range(1,amount_of_diferent_teams+1), tablefmt='fancy_grid'))

╒════╤══════════════════════════╤══════════════════════════╤═══════════════════════════════════════════╤═══════════════════════════╤═══════════════════════╤════════════════════════╤══════════════════════════════════════════╕
│    │ Team                     │   Total amount of points │   Number of seasons in the Premier League │   % of victories as local │   % of draws as local │   % of losses as local │   % of goals scored vs conceded as local │
╞════╪══════════════════════════╪══════════════════════════╪═══════════════════════════════════════════╪═══════════════════════════╪═══════════════════════╪════════════════════════╪══════════════════════════════════════════╡
│  1 │ Manchester United        │                      544 │                                        12 │                        75 │                    14 │                     11 │                                     3.09 │
├────┼──────────────────────────┼──────────────────────────┼────────────────────────────────────────

# Conclusiones

El mejor equipo jugando como local es el Manchester United tanto en término de puntos acumulados como en tasa de victoria como en proporción de goles marcados frente a recibidos. Asimismo, los otros 6 equipos por debajo, es decir, desde el Chealse hasta el Everton, destacan frente al resto por varios factores:
 1. La diferencia de puntos con el equipo siguiente en la tabla, el West Ham City, es de 119 puntos.
 2. Estos 6 equipos al igual que el Manchester City, son los únicos que han estado las 12 temporadas analizadas en primera.
 3. Su tasa de victorias jugando en casa es mayor a la suma de la tasa de empates y de derrotas. 
 
Sin embargo, aunque hay una relación directamente proporcional entre estas variables, destacan equipos como Sheffield United o el Birmingham City que a pesar de tener relativamente pocos puntos acumulados jugando como locales, tienen una proporción de goles por encima de 1, es decir, han marcado más goles que los que han encajado en su campo.

De igual forma, llama la atención que en la tabla tenemos más de 20 equipos que forman la Premier League, esto es debido a que cada año 3 equipos bajan a seguna y 3 suben a primera. Es por ello también, que se ha considerado relevante incluir una columna en la que se indique el número de temporadas en primera.

Por último, comentar que la diferencia de realizar esta misma tarea con Python con respecto a PySpark es la escalabilidad que nos permite Spark para realizar esta misma tarea, por ejemplo, con un dataset que contuviera todo el histórico de la Liga Inglesa.

